In [7]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
    
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

    
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [38]:

start = 'https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=All&page={}'.format(0)
soup = connect_to_page(start)

In [ ]:
today = date.today().strftime("%Y-%m-%d")
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded",])
form_dest = "PDFS/NC_forms/All_forms/"
# count = 0

52 pages in total (from 0 to 51)

In [ ]:
# articles = soup.find_all("article")
pdfs_href = {}

for count in range(52):
    time.sleep(2)
    start = 'https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=All&page={}'.format(count)
    soup = connect_to_page(start)
    articles = soup.find_all("article")
#--------------------------------------------Forms from each page ----------------------------------------
    for i in articles:
        meta = i.find("div", class_ = "meta")
        a = i.find("a")
        
        if "www.nccourts.gov/documents" in a["href"]:
            
            url = a["href"]
            filename = meta.text.split(",")[0].replace(";","") + '.pdf'

            if filename not in pdfs_href:
                pdfs_href[filename] = url
                group = re.sub(r"\([^()]*\)","",meta.text.split(",")[1])
                print(a['href'], filename,group, a.text)


In [ ]:
BASE = "https://www.nccourts.gov"
today = date.today().strftime("%Y-%m-%d")
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","file_format"])
form_dest = "PDFS/NC_forms/All_forms/"
source = "https://www.nccourts.gov/documents/forms"
jur = 'NC'
pdfs_href = {}

for count in range(52):
    time.sleep(1)
    start = 'https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=All&page={}'.format(count)
    soup = connect_to_page(start)
    articles = soup.find_all("article")


    for cat in articles:
        time.sleep(1)
        meta = cat.find("div", class_ = "meta")
        a = cat.find("a")
            
        if "www.nccourts.gov/documents" in a["href"]:
                
            url = a["href"]
            filename = (meta.text.split(",")[0].replace(";","") + '.pdf').strip()
            filename = re.sub(r'\s',"-", filename)

            if filename not in pdfs_href:
                pdfs_href[filename] = url
                group = re.sub(r"\([^()]*\)","",meta.text.split(",")[1])
                # print(url, filename,group, a.text)

                soup_form_page = connect_to_page(url)
                a_form_page = soup_form_page.find_all("a", itemprop= True)
                    
                for pdf in a_form_page:
                    title = pdf.text.strip()
                    
                    url_form = BASE+pdf['href'].strip()
                    file_id = hashme(url_form)
                    status = download_pdf(url_form, form_dest,file_id+'.pdf')
                    print(title,filename, url_form)
                    if status=='error':
                        print("##########################################")
                        print("ERROR")
                        print("##########################################")
                        if ".doc" in url_form:
                            download_docx(url_form,file_id+".doc")
                            file_format = "doc"
                            files_df = files_df.append(pd.DataFrame([[file_id,jur,source,title,group,url_form,filename,today, file_format]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","file_format"]))
                    else:
                        file_format = 'PDF'
                        files_df = files_df.append(pd.DataFrame([[file_id,jur,source,title,group,url_form,filename,today, file_format]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","file_format"]))
                
files_df.to_csv("NC_metaData-V2.csv",index=False, encoding="utf-8")

In [17]:
BASE = "https://www.nccourts.gov"
today = date.today().strftime("%Y-%m-%d")
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","file_format"])
form_dest = "PDFS/NC_forms/All_forms/"
source = "https://www.nccourts.gov/documents/forms"
jur = 'NC'
pdfs_href = {}

for count in range(52):
    time.sleep(1)
    start = 'https://www.nccourts.gov/documents/forms?field_form_type_target_id=All&field_language_target_id=All&page={}'.format(count)
    soup = connect_to_page(start)
    articles = soup.find_all("article")


    for cat in articles:
        time.sleep(1)
        meta = cat.find("div", class_ = "meta")
        a = cat.find("a")
            
        if "www.nccourts.gov/documents" in a["href"]:
                
            url = a["href"]
            filename = (meta.text.split(",")[0].replace(";","") + '.pdf').strip()
            filename = re.sub(r'\s',"-", filename)

            if filename not in pdfs_href:
                pdfs_href[filename] = url
                group = re.sub(r"\([^()]*\)","",meta.text.split(",")[1])
                # print(url, filename,group, a.text)

                soup_form_page = connect_to_page(url)
                a_form_page = soup_form_page.find_all("a", itemprop= True)
                    
                for pdf in a_form_page:
                    title = pdf.text.strip()
                    
                    url_form = BASE+pdf['href'].strip()
                    if ".doc" in url_form:
                        file_id = hashme(url_form)
                        print(title,filename, url_form)  
                        download_docx(url_form,file_id+".doc")
                        file_format = "doc"
                        files_df = files_df.append(pd.DataFrame([[file_id,jur,source,title,group,url_form,filename,today, file_format]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded","file_format"]))
                    
                
files_df.to_csv("NC_metaData-docs.csv",index=False, encoding="utf-8")

Position Description AOC-A-114.pdf https://www.nccourts.gov/assets/documents/forms/767.docx?QBo.4PrhFNKkTifwznfshh6PZwIu82iY
Notice Of Returned Check Form - Motor Vehicle AOC-FS-1510.pdf https://www.nccourts.gov/assets/documents/forms/aoc-fs-1510-en.doc?lhJhinvF9TGflBJuTwbpWuGj8atsR2pL
Notice Of Returned Check Form - Non-Motor Vehicle AOC-FS-1511.pdf https://www.nccourts.gov/assets/documents/forms/aoc-fs-1511-en.doc?23cEYuySBbYa10iDMfq9XlmC7ll3AmCw
